Import necessary libraries
authenticate using gcloud api enabled on gcloud project

In [131]:
# 1 import libraries
import ee
import geopandas as gpd
import geemap
import pandas as pd
import folium
import geemap.foliumap as geemap
from datetime import datetime, timedelta
import os


Trigger authentication

In [132]:

# run this in terminal to authenticate with gcloud >> earthengine authenticate

Initialize Earth Engine

In [133]:


ee.Initialize()

# Parameters
surface reflectance threshold
start date
the end date is currently start date + 15 days and the first clear scene, low cloud cover will be selected and the image date is used in the reporting and filenaming convention along with the SR threshold value.

Spring 2023

In [134]:
#---Mar 2023---730 ac***
# threshold = .15
# start_date = '2023-02-15'

#---Mar 2023---1604 ac
# threshold = .20
# start_date = '2023-02-15'

#---May 2023---300 ac
# threshold = .15
# start_date = '2023-05-01'

#---May 2023---449 ac
# threshold = .20
# start_date = '2023-05-01'


Fall 2023

In [135]:

#---Oct 2023---1175 ac
# threshold = .22
# start_date = '2023-10-03'

#---Nov 2023---618 ac
# threshold = .16
# start_date = '2023-11-03'

#---Nov 2023---760 ac
# threshold = .18
# start_date = '2023-11-03'

#---Nov 2023---987 ac
# threshold = .20
# start_date = '2023-11-03'

#---Nov 2023--- 1125 ac
threshold = .21
start_date = '2023-11-03'

#---Nov 2023---1325 ac***
# threshold = .22
# start_date = '2023-11-03'




Spring 2024

In [136]:

#---Mar 2024---1130 acres***
# threshold = .15
# start_date = '2024-02-25'

#---Mar 2024---1355 acres***
# threshold = .16
# start_date = '2024-02-25'

#---May 2024---
# start_date = '2024-05-10'
# threshold = .22

#---Jun 2024---92 ac
# start_date = '2024-06-01'
# threshold = .15

#---Jun 2024---109 ac
# start_date = '2024-06-01'
# threshold = .16

#---Jun 2024---109 ac
# start_date = '2024-06-06'
# threshold = .16


Compute the end date by adding days to the start date

In [137]:


end_date = (datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=15)).strftime('%Y-%m-%d')


Report Directory

Define the bounding box coordinates and create a bounding box geometry

In [138]:
# Define the bounding box coordinates

bbox = [[-118.23240736400778, 36.84651455123723],
        [-118.17232588207419, 36.84651455123723],
        [-118.17232588207419, 36.924364295139625],
        [-118.23240736400778, 36.924364295139625]]

# Create a bounding box geometry
bounding_box_geometry = ee.Geometry.Polygon(bbox)


Filter sentinel 2 surface reflectance imagery and extract dates, pixel size

In [139]:
# COPERNICUS/S2_SR_HARMONIZED, allows comparisons between dates compared to S2_SR
sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(bounding_box_geometry) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
    .select(['B8', 'SCL'])  # Select NIR band (B8) and Scene Classification Layer (SCL) for masking clouds


In [140]:

size = sentinel_collection.size().getInfo()
print(f"Number of images in collection: {size}")



Number of images in collection: 4


In [141]:

# Select a single band before retrieving the projection information
pixel_size = sentinel_collection.first().select('B8').projection().nominalScale().getInfo()


In [142]:

# Extract the date information
image_info = sentinel_collection.first().getInfo()
image_date = image_info['properties']['system:time_start']
image_date_str = pd.to_datetime(image_date, unit='ms').strftime('%Y-%m-%d')



report and map directories and file names

In [143]:
# directory location for reports
html_subdirectory = "flood_reports/html_reports_maps"
os.makedirs(html_subdirectory, exist_ok=True)


# Define filenames with unique names based on the image date
report_filename = os.path.join(html_subdirectory, f"bwma_flood_report_{image_date_str}_{threshold}.html")
map_filename = os.path.join(html_subdirectory, f"flooded_area_map_{image_date_str}_{threshold}.html")


Create a cloud-free composite by masking cloudy pixels

In [144]:



def mask_clouds(image):
    cloud_prob = image.select('SCL')  # Scene Classification Layer (SCL)
    is_cloud = cloud_prob.eq(9)  # 9 represents clouds in the SCL
    return image.updateMask(is_cloud.Not())

cloud_free_composite = sentinel_collection.map(mask_clouds).median()

Define binary image base on the threshold

In [145]:
binary_image = cloud_free_composite.select('B8').divide(10000).lt(threshold).selfMask()

Load units from geojson and convert to earth engine geometry

In [146]:
gdf = gpd.read_file("data/unitsBwma2800.geojson")
units = geemap.geopandas_to_ee(gdf)
units = units.filterBounds(bounding_box_geometry)
units_clipped = units.map(lambda feature: feature.intersection(bounding_box_geometry))

Compute the unit area and flooded area

In [147]:
# Compute area of each unit
def compute_area(feature):
    return feature.set({'unit_acres': feature.geometry().area().divide(4046.86)})

units_with_area = units_clipped.map(compute_area)

# Compute the flooded area within each unit
def compute_refined_flood_area(feature):
    geom = feature.geometry()
    total_pixels = cloud_free_composite.select('B8').reduceRegion(
        reducer=ee.Reducer.count(), geometry=geom, scale=10).get('B8')
    flooded_pixels = binary_image.reduceRegion(
        reducer=ee.Reducer.count(), geometry=geom, scale=10).get('B8')
    pixel_area_m2 = ee.Number(pixel_size).multiply(pixel_size)
    pixel_area_acres = pixel_area_m2.multiply(0.000247105)
    flooded_area_acres = ee.Number(flooded_pixels).multiply(pixel_area_acres)
    flooded_percentage = ee.Number(flooded_pixels).divide(total_pixels).multiply(100)
    unit_name = feature.get('Flood_Unit')
    label = ee.String(unit_name).cat(': ').cat(flooded_area_acres.format('%.2f')).cat(' Acres')
    return feature.set({
        'total_pixels': total_pixels,
        'flooded_pixels': flooded_pixels,
        'acres_flooded': flooded_area_acres,
        'flooded_percentage': flooded_percentage,
        'label': label
    })

units_with_calculations = units_with_area.map(compute_refined_flood_area)

Convert EE feature collection to Pandas DataFrame

In [148]:

units_df_properties_reduced = pd.DataFrame(units_with_calculations.getInfo()['features'])
units_df_properties_reduced = pd.json_normalize(units_df_properties_reduced['properties'])
units_df_properties_reduced = units_df_properties_reduced[['Flood_Unit', 'total_pixels', 'flooded_pixels', 'unit_acres', 'acres_flooded', 'flooded_percentage']]

# Round numbers
units_df_properties_reduced = units_df_properties_reduced.round(2)

Calculate Total Acreage and add total row

In [149]:
# Calculate totals
total_acres = units_df_properties_reduced['unit_acres'].sum()
total_flooded_acres = units_df_properties_reduced['acres_flooded'].sum()
total_flooded_percentage = (total_flooded_acres / total_acres) * 100

# Add a row for totals
totals = pd.DataFrame([{
    'Flood_Unit': 'Total',
    'total_pixels': units_df_properties_reduced['total_pixels'].sum(),
    'flooded_pixels': units_df_properties_reduced['flooded_pixels'].sum(),
    'unit_acres': total_acres,
    'acres_flooded': total_flooded_acres,
    'flooded_percentage': total_flooded_percentage
}])

# Concatenate totals to the original dataframe
units_df_properties_reduced = pd.concat([units_df_properties_reduced, totals], ignore_index=True)

# Round the totals row
units_df_properties_reduced = units_df_properties_reduced.round(2)

# Display the formatted DataFrame
print(units_df_properties_reduced.to_string(index=False))



     Flood_Unit  total_pixels  flooded_pixels  unit_acres  acres_flooded  flooded_percentage
        Thibaut         99955           16952     1971.07         418.89               16.96
           Drew         25462            3544      502.65          87.57               13.92
      Waggonner         63198           12079     1246.13         298.48               19.11
 West Winterton         40855            3872      805.68          95.68                9.48
 East Winterton         17010            2391      335.14          59.08               14.06
South Winterton         26035            4390      513.46         108.48               16.86
  Thibaut Ponds         26753            2311      527.63          57.11                8.64
          Total        299268           45539     5901.76        1125.29               19.07


Create and Save HTML Map

In [150]:
# Clip the binary image to the unit boundaries
clipped_binary_image = binary_image.clip(units)

# Create a folium map
Map = geemap.Map(center=[36.8795, -118.202], zoom=12)

# Add the clipped binary image to the map
Map.addLayer(clipped_binary_image, {'palette': ['blue'], 'opacity': 0.5}, 'Flooded Pixels')

# Style for unit boundaries
units_style = {'color': 'red', 'fillColor': '00000000'}  # '00000000' is transparent in RGBA

# Add the unit boundaries to the map
Map.addLayer(units_with_calculations.style(**units_style), {}, 'Unit Boundaries')

# Add labels to the map
labels = units_with_calculations.aggregate_array('label').getInfo()

# Extract centroids for each feature
def get_centroid(feature):
    return feature.geometry().centroid().coordinates()

centroids = units_with_calculations.map(lambda f: f.set('centroid', get_centroid(f)))

# Get the centroids information
centroid_info = centroids.aggregate_array('centroid').getInfo()

for label, centroid in zip(labels, centroid_info):
    folium.Marker(
        location=[centroid[1], centroid[0]],
        icon=None,
        popup=label
    ).add_to(Map)

# Display the map
Map.add_child(folium.LayerControl())
# Save the map to the subdirectory with a unique name
Map.save(map_filename)
Map

Write csv Table

In [151]:

# Define the subdirectory for CSV output
csv_subdirectory = "flood_reports/csv_output"
os.makedirs(csv_subdirectory, exist_ok=True)

# Save the DataFrame to a CSV file
units_df_properties_reduced = pd.DataFrame(units_with_calculations.getInfo()['features'])
units_df_properties_reduced = pd.json_normalize(units_df_properties_reduced['properties'])
units_df_properties_reduced = units_df_properties_reduced[['Flood_Unit', 'total_pixels', 'flooded_pixels', 'unit_acres', 'acres_flooded']]

# Define the CSV filename with the new directory
csv_filename = os.path.join(csv_subdirectory, f'flood_report_data_{image_date_str}_{threshold}.csv')

# csv_filename = f'flood_report_data_{image_date_str}.csv'
units_df_properties_reduced.to_csv(csv_filename, index=False)


Create HTML Table

In [152]:

# Rename columns for display
units_df_properties_reduced = units_df_properties_reduced[['Flood_Unit', 'acres_flooded']]
units_df_properties_reduced.columns = ['BWMA Unit', 'Acres']

# Round acres to whole numbers
units_df_properties_reduced['Acres'] = units_df_properties_reduced['Acres'].round(0).astype(int)

# Calculate totals
total_acres = units_df_properties_reduced['Acres'].sum()
totals_row = pd.DataFrame([{'BWMA Unit': 'Total', 'Acres': total_acres}])

# Append the totals row to the DataFrame
units_df_properties_reduced = pd.concat([units_df_properties_reduced, totals_row], ignore_index=True)


# Style the DataFrame and convert to HTML
html_table_simple = (
    units_df_properties_reduced.style
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('font-size', '14px')]},
        {'selector': 'td:nth-child(1)', 'props': [('text-align', 'left'), ('font-size', '12px')]},
        {'selector': 'td:nth-child(2)', 'props': [('text-align', 'right'), ('font-size', '12px')]},
    ])
    .set_properties(subset=pd.IndexSlice[units_df_properties_reduced.index[-1], :], **{'font-weight': 'bold', 'font-size': '14px'})
    .hide(axis='index')
    .to_html()
)


# Manually hide the index in the HTML string
html_table_simple = html_table_simple.replace('<th></th>', '')


Create HTML Report

In [153]:


# HTML report
html_report = f"""
<html>
<head>
    <title>BWMA Flooded Extent: {image_date_str}</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
        }}
        .container {{
            display: flex;
            justify-content: space-between;
        }}
        .left {{
            width: 25%;
        }}
        .right {{
            width: 75%;
            text-align: center;
        }}
        h1, h2 {{
            text-align: center;
        }}
        .notes {{
            margin-top: 20px;
            padding: 10px;
            border-top: 1px solid #000;
        }}
        /*----CHANGE---- Increase font size and add line above totals row */
        table {{
            width: 90%;
            font-size: 50px;
            border-collapse: collapse;
        }}
        th, td {{
            padding: 20px;
            text-align: left;
            border-bottom: 1px solid #ddd;
        }}
        th {{
            text-align: right;//from center
            font-size: 30px;
        }}
        td:nth-child(2) {{
            text-align: right;
        }}
        tr:last-child {{
            font-weight: bold;
            font-size: 80px;
        }}
        tr:last-child td {{
            border-top: 3px solid #000;
        }}
    </style>
</head>
<body>
    <h1>BWMA Flooded Acres and Extent: {image_date_str}</h1>
    <div class="container">
        <div class="left">
            <h2>Flooded Acres</h2>
            {html_table_simple}
        </div>
        <div class="right">
            <h2>Spatial Extent</h2>
            <p>Imagery Date: {image_date_str}</p>
            <iframe src="flooded_area_map_{image_date_str}_{threshold}.html" width="90%" height="500"></iframe>
        </div>
    </div>
    <div class="notes">
        <h3>Technical Notes</h3>
        <p>Flooded acres were calculated from Sentinel-2 Surface Reflectance imagery using the Earth Engine Python API in a Jupyter notebook.  Sentinel-2 (S2) is a wide-swath, high-resolution, multispectral imaging mission with a global 5-day revisit frequency.</p>
        <p>The S2 Multispectral Instrument (MSI) samples 13 spectral bands: Visible and NIR at 10 meters, red edge and SWIR at 20 meters, and atmospheric bands at 60 meters spatial resolution. The Near Infrared (NIR) band was used to identify flooded areas by applying a threshold to isolate water.</p>
        <p>The results are validated during routine field checks throughout the seasonal flooding cycle September through April.</p>
    </div>
</body>
</html>
"""

# Save the report to an HTML file in the subdirectory
with open(report_filename, "w") as file:
    file.write(html_report)

print(f"Report saved to {report_filename}")
print(f"Map saved to {map_filename}")


Report saved to flood_reports/html_reports_maps\bwma_flood_report_2023-11-09_0.21.html
Map saved to flood_reports/html_reports_maps\flooded_area_map_2023-11-09_0.21.html
